This code block is responsible for training and evaluating a Vanilla RNN for EEG signal classification and measuring test accuracy.

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import numpy as np

# Define the Vanilla RNN model
class VanillaRNN(nn.Module):
    def __init__(self, input_size=22, hidden_size=64, output_size=4, num_layers=2):
        super(VanillaRNN, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers

        self.rnn = nn.RNN(input_size, hidden_size, num_layers, batch_first=True, nonlinearity='tanh')
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        out, _ = self.rnn(x, h0)
        out = self.fc(out[:, -1, :])  # Use the last time step's output for classification
        return out

# Generate Synthetic EEG Data
def generate_synthetic_data(num_samples=1000, seq_length=20, input_size=22, num_classes=4):
    X = np.random.rand(num_samples, seq_length, input_size)
    y = np.random.randint(0, num_classes, num_samples)
    return torch.tensor(X, dtype=torch.float32), torch.tensor(y, dtype=torch.long)

# Prepare dataset
X_train_valid, y_train_valid = generate_synthetic_data(num_samples=800)
X_test, y_test = generate_synthetic_data(num_samples=200)

batch_size = 32
train_dataset = TensorDataset(X_train_valid, y_train_valid)
test_dataset = TensorDataset(X_test, y_test)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# Training function
def train_rnn(num_epochs=40, learning_rate=0.001):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = VanillaRNN().to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)

    for epoch in range(num_epochs):
        model.train()
        total_loss = 0
        correct = 0
        total = 0

        for batch_X, batch_y in train_loader:
            batch_X, batch_y = batch_X.to(device), batch_y.to(device)
            optimizer.zero_grad()
            outputs = model(batch_X)
            loss = criterion(outputs, batch_y)
            loss.backward()
            optimizer.step()

            total_loss += loss.item()
            _, predicted = torch.max(outputs, 1)
            correct += (predicted == batch_y).sum().item()
            total += batch_y.size(0)

        train_accuracy = 100 * correct / total
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {total_loss / len(train_loader):.4f}, Train Accuracy: {train_accuracy:.2f}%')

    # Evaluate the model
    model.eval()
    correct = 0
    total = 0

    with torch.no_grad():
        for batch_X, batch_y in test_loader:
            batch_X, batch_y = batch_X.to(device), batch_y.to(device)
            outputs = model(batch_X)
            _, predicted = torch.max(outputs, 1)
            correct += (predicted == batch_y).sum().item()
            total += batch_y.size(0)

    test_accuracy = 100 * correct / total
    print(f'Test Accuracy: {test_accuracy:.2f}%')
    return model

# Train the Vanilla RNN
rnn_model = train_rnn()


Epoch [1/40], Loss: 1.3996, Train Accuracy: 24.12%
Epoch [2/40], Loss: 1.3820, Train Accuracy: 26.00%
Epoch [3/40], Loss: 1.3676, Train Accuracy: 34.25%
Epoch [4/40], Loss: 1.3491, Train Accuracy: 35.50%
Epoch [5/40], Loss: 1.3222, Train Accuracy: 39.25%
Epoch [6/40], Loss: 1.2966, Train Accuracy: 39.75%
Epoch [7/40], Loss: 1.2611, Train Accuracy: 42.12%
Epoch [8/40], Loss: 1.2324, Train Accuracy: 47.12%
Epoch [9/40], Loss: 1.2023, Train Accuracy: 45.75%
Epoch [10/40], Loss: 1.1763, Train Accuracy: 50.50%
Epoch [11/40], Loss: 1.1474, Train Accuracy: 50.62%
Epoch [12/40], Loss: 1.1338, Train Accuracy: 49.62%
Epoch [13/40], Loss: 1.1102, Train Accuracy: 52.12%
Epoch [14/40], Loss: 1.0969, Train Accuracy: 53.38%
Epoch [15/40], Loss: 1.0736, Train Accuracy: 55.00%
Epoch [16/40], Loss: 1.0448, Train Accuracy: 55.00%
Epoch [17/40], Loss: 1.0277, Train Accuracy: 57.62%
Epoch [18/40], Loss: 1.0023, Train Accuracy: 57.88%
Epoch [19/40], Loss: 0.9886, Train Accuracy: 58.88%
Epoch [20/40], Loss: 

This code block extends the first by adding training time measurement.

In [ ]:
import time
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import numpy as np

# Define the Vanilla RNN model
class VanillaRNN(nn.Module):
    def __init__(self, input_size=22, hidden_size=64, output_size=4, num_layers=2):
        super(VanillaRNN, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers

        self.rnn = nn.RNN(input_size, hidden_size, num_layers, batch_first=True, nonlinearity='tanh')
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        out, _ = self.rnn(x, h0)
        out = self.fc(out[:, -1, :])  # Use the last time step’s output for classification
        return out

# Generate Synthetic EEG Data
def generate_synthetic_data(num_samples=1000, seq_length=20, input_size=22, num_classes=4):
    X = np.random.rand(num_samples, seq_length, input_size)
    y = np.random.randint(0, num_classes, num_samples)
    return torch.tensor(X, dtype=torch.float32), torch.tensor(y, dtype=torch.long)

# Prepare dataset
X_train_valid, y_train_valid = generate_synthetic_data(num_samples=800)
X_test, y_test = generate_synthetic_data(num_samples=200)

batch_size = 32
train_dataset = TensorDataset(X_train_valid, y_train_valid)
test_dataset = TensorDataset(X_test, y_test)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# Training function with time measurement
def train_rnn(num_epochs=40, learning_rate=0.001):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = VanillaRNN().to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)

    # Start time measurement
    start_time = time.time()

    for epoch in range(num_epochs):
        model.train()
        total_loss = 0
        correct = 0
        total = 0

        for batch_X, batch_y in train_loader:
            batch_X, batch_y = batch_X.to(device), batch_y.to(device)
            optimizer.zero_grad()
            outputs = model(batch_X)
            loss = criterion(outputs, batch_y)
            loss.backward()
            optimizer.step()

            total_loss += loss.item()
            _, predicted = torch.max(outputs, 1)
            correct += (predicted == batch_y).sum().item()
            total += batch_y.size(0)

        train_accuracy = 100 * correct / total
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {total_loss / len(train_loader):.4f}, Train Accuracy: {train_accuracy:.2f}%')

    # End time measurement
    end_time = time.time()
    training_time = end_time - start_time
    print(f"Total Training Time: {training_time:.2f} seconds ({training_time/60:.2f} minutes)")

    return model, training_time

# Train the Vanilla RNN and measure training time
rnn_model, training_time = train_rnn()


Epoch [1/40], Loss: 1.3914, Train Accuracy: 25.12%
Epoch [2/40], Loss: 1.3758, Train Accuracy: 30.25%
Epoch [3/40], Loss: 1.3607, Train Accuracy: 31.88%
Epoch [4/40], Loss: 1.3376, Train Accuracy: 35.12%
Epoch [5/40], Loss: 1.3016, Train Accuracy: 39.88%
Epoch [6/40], Loss: 1.2621, Train Accuracy: 45.00%
Epoch [7/40], Loss: 1.2326, Train Accuracy: 42.75%
Epoch [8/40], Loss: 1.1949, Train Accuracy: 48.12%
Epoch [9/40], Loss: 1.1647, Train Accuracy: 48.75%
Epoch [10/40], Loss: 1.1349, Train Accuracy: 51.50%
Epoch [11/40], Loss: 1.1202, Train Accuracy: 51.62%
Epoch [12/40], Loss: 1.1058, Train Accuracy: 51.88%
Epoch [13/40], Loss: 1.0798, Train Accuracy: 55.00%
Epoch [14/40], Loss: 1.0652, Train Accuracy: 55.12%
Epoch [15/40], Loss: 1.0452, Train Accuracy: 56.75%
Epoch [16/40], Loss: 1.0310, Train Accuracy: 59.50%
Epoch [17/40], Loss: 1.0018, Train Accuracy: 60.25%
Epoch [18/40], Loss: 0.9869, Train Accuracy: 61.12%
Epoch [19/40], Loss: 0.9680, Train Accuracy: 61.88%
Epoch [20/40], Loss: 